In [1]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
import numpy as np
import cv2
import matplotlib as mpl
import matplotlib.pyplot as plt
import subprocess

sys.path.append('fbsource/fbcode/scripts/psarlin/')
from maploc.utils.viz_2d import plot_images, plot_keypoints, add_text, save_plot
from maploc.utils.geo import Projection
from maploc.data.aria import main, datasets, query_gaia_ids

In [ ]:
!cd fbsource/fbcode && buck build @mode/opt -c python.package_style=inplace surreal/aria_research_tools:run_dfm2
!cd fbsource/fbcode && buck build @mode/opt -c python.package_style=inplace surreal/aria_research_tools:run_aria2kapture

In [2]:
# gaia_ids = query_gaia_ids("surreal-aria-reloc-seattle-downtown")
# projection = Projection("EPSG:32148")  # seattle area
# scene = "reloc_seattle_downtown"

# gaia_ids = query_gaia_ids("surreal-aria-reloc-detroit-greektown")
# projection = Projection("EPSG:6498")  # detroit area
# scene = "reloc_detroit_greektown"

# gaia_ids = query_gaia_ids("surreal-aria-reloc-detroit-gcp")
# projection = Projection("EPSG:6498")  # detroit area
# scene = "reloc_detroit_gcp"

gaia_ids = query_gaia_ids("surreal-aria-reloc-detroit-cp")
projection = Projection("EPSG:6498")  # detroit area
scene = "reloc_detroit_cp"

output_dir = Path("/home/psarlin/local/data/aria_dumps/", scene)
tmp_dir = Path("/home/psarlin/local/aria")
output_dump, all_xy, tile_manager = main(output_dir, gaia_ids, projection, tmp_dir)

## Visualizations

In [23]:
centers = []
for ts in timestamps_rgb:
    T_w2rgb = traj[int(ts), sensor_id_rgb]
    T_rgb2w = T_w2rgb.inverse()
    centers.append(T_rgb2w.t.squeeze(-1))
centers = np.array(centers)
dists = np.linalg.norm(centers[1:]-centers[:-1], axis=-1)
np.sum(dists)

In [ ]:
n = 150
plt.figure(dpi=200)
from maploc.utils.viz_localization import plot_pose
plt.scatter(*xy[:n].T, c=np.linspace(0, 1, len(xy[:n])), cmap="jet", s=1)
# [plot_pose(0, uv-.5, y, s=1/3, c="g", w=0.015) for i, (uv, y) in enumerate(zip(xy[:n], yaw[:n]))];
uv = (np.sin(np.deg2rad(yaws)), np.cos(np.deg2rad(yaws)))
plt.gca().quiver(
    *xy[:n].T,
    uv[0][:n], uv[1][:n],
    scale=1/3,
    scale_units="xy",
    angles="xy",
    color="k",
    zorder=1,
    alpha=1,
    width=0.005
)
plt.gca().set_aspect("equal")

In [ ]:
from maploc.utils.viz_3d import init_figure, plot_camera, plot_points

fig = init_figure()
# plot_points(fig, p3d)
K = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])

xy = []
yaws = []
for idx in tqdm(range(len(timestamps_rgb_select))):
        ts = timestamps_rgb_select[idx]
        T_w2rgb = traj[int(ts), sensor_id_rgb]
        T_rgb2w = T_w2rgb.inverse()
        t_c2w = T_rgb2w.t.squeeze(-1)
        R_c2w = quaternion.as_rotation_matrix(T_rgb2w.r)
        rpy = decompose_rotmat(R_c2w)
        latlon = ts2latlon[ts_rgb2gps[ts]]
        plot_camera(fig, R_c2w, t_c2w, K)
        yaws.append(rpy[-1])
        xy.append(t_c2w[:2])
xy = np.array(xy)
yaws = np.array(yaws)
fig.show()